- Load a few different data sources and look for tokens before 'ord'

In [3]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
# from torchmetrics.regression import KendallRankCorrCoef, SpearmanCorrCoef
from collections import defaultdict

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import get_mlp_activations
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [4]:
haystack_utils.clean_cache()
model = HookedTransformer.from_pretrained("EleutherAI/pythia-160m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-160m into HookedTransformer


In [5]:
from datasets import load_dataset
dataset = load_dataset("NeelNanda/pile-10k", split='train')
print(dataset[0])

{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a species facing a challenge.

In [6]:
stack_exchange_data = load_dataset('habedi/stack-exchange-dataset', split='train')

In [9]:
F_prompts = []
for prompt in stack_exchange_data:
    prompt = prompt['body']
    tokens = model.to_str_tokens(model.to_tokens(prompt))
    if 'ord' in tokens:
        ord_index = tokens.index('ord')
        if ord_index > 0 and 'F' in tokens[ord_index - 1]:
            F_prompts.append((prompt, ord_index))

In [11]:
import json

print(len(F_prompts))
with open('data/F_prompts.json', 'w') as file:
    json.dump([prompt[0] for prompt in F_prompts], file)

118


In [ ]:
from collections import Counter
record_prompts = []

counter = Counter()
for prompt in dataset:
    prompt = prompt['text']
    tokens = model.to_str_tokens(model.to_tokens(prompt))
    if 'ord' in tokens:
        ord_index = tokens.index('ord')
        if ord_index > 0: 
            counter.update(tokens[ord_index - 1])
            # print(''.join(tokens[ord_index - 8:ord_index+1]))
            # print(prompt[prompt.index('ord') - 10 : prompt.index('ord') + 50])
        if ord_index > 0 and 'rec' in tokens[ord_index - 1]:
            record_prompts.append(prompt, ord_index)
            print(True)

KeyboardInterrupt: 

In [ ]:
print(counter)

Counter({' ': 49, 'f': 29, 'c': 16, 'i': 15, 'e': 10, 'a': 10, 'h': 10, 'o': 9, 'l': 7, '-': 7, 'r': 7, 'b': 6, 'F': 6, 'J': 6, 'G': 6, 's': 6, 'A': 5, 'C': 5, 'n': 5, 'H': 4, 'N': 4, 'w': 3, 'j': 3, 'd': 3, 'B': 3, 'u': 3, 'v': 3, 'm': 2, 'D': 2, 'g': 2, 'R': 2, 'T': 2, 'M': 2, "'": 1, 'x': 1, 't': 1, '\n': 1, '’': 1, 'L': 1, 'S': 1, 'I': 1})


In [ ]:

# prompts = ['order', 'ordinal', 'ordinary', 'ordain']
# record_prompts = []
# for prompt in dataset:
#     tokens = model.to_str_tokens(model.to_tokens(prompt))
#     if 'ord' in tokens:
#         ord_index = tokens.index('ord')
#         if ord_index > 0: print(tokens[ord_index - 1])
#         if ord_index > 0 and 'rec' in tokens[ord_index - 1]:
#             record_prompts.append(prompt, ord_index)
#             print(True)
#             print(prompt[prompt.index('ord') - 10 : prompt.index('ord') + 50])


# for prompt in prompts:
#     print(model.to_str_tokens(prompt))